# Callable이란?

<p> Callable은 '호출가능한'이라는 뜻이며, 파이썬의 <b>함수</b> 또는 함수화된 호출가능한 클래스를 의미합니다.</p>
<p> Callable 클래스의 기본예시를 먼저 보여드리겠습니다.</p>

In [1]:
class Hi(object):
    def __init__(self):
        pass

    # 클래스 내 __call__ 메소드를 정의하면 Callable한 객체가 됨.
    def __call__(self):
        message = '안녕!'
        return message
    
hi = Hi()
hi() # 클래스를 호출할 수 있음.

'안녕!'

# 용례
<p> 이걸 어디에 쓰는지 궁금하실 겁니다. 쓸 곳이 있기나 할까?</p>

## 1. Decorator
우선, 파이썬에 '장식자'라는 개념이 있습니다. 뭔지 코드로 보시죠 :

In [29]:
from time import time

# -- 장식자 클래스
class Timer(object):
    def __init__(self, func):
        self.func = func                  # 1. 장식자는 장식되는 함수를 인자로 받는다.

    def __call__(self, *args, **kwargs): 
        stt = time()                      # 2. 장식되는 함수의 앞에서 뭔가를 해준다.
        self.func(*args, **kwargs)        # 3. 장식되는 함수를 실행한다.
        end = time()                      # 4. 장식되는 함수의 뒤에서 뭔가를 한다.
        print('{0:.2f} secs'.format(end - stt))

call 함수를 정의했으니, 장식자 클래스도 callable이죠? 장식자에 대해 간단히 설명만 하고 넘어갈게요.

In [3]:
@Timer
def hello():
    _t = [1 for i in range(1000000)]
    print('hello!')
    return

hello()

hello!
0.06 secs


<b>오잉? 함수 정의에서는 hello!만 출력했는데, 실행시간까지 출력</b>됐네요. <br>이것이 장식자의 기능입니다.
잘 보면 함수정의 위에, '@장식자클래스'가 붙어있어요. <br>이렇게 Decorator는 보통 함수 앞뒤에 똑같은 기능을 계속 넣어줘야할 때 아예 클래스로 빼서, 중복코드를 최소화시키는 전략으로 많이 사용됩니다.<br>
<br>
프로그래머는 장식자를 활용하면 코드 작성 시간도 줄이고, 좋은 코드도 되고 1석 2조의 효과를 거둡니다.<br>
단점이라면, 장식자를 모르는 사람이 코드를 볼때 헷갈릴 수도 있다는 것이겠는데요. 단점아닌 단점이겠죠.

## 2. 인자로 활용

파이썬에서는 함수 자체가 인자로 전달될 수 있습니다. 이러한 특징 덕분에, 파이썬의 많은 유명한 패키지 (scikit-learn, spaCy 등)에서 callable을 인자로 받는데요. 예시를 하나 보겠습니다 :

In [35]:
from spacy import load
from spacy.tokens import Doc

if __name__ == '__main__':
    raw = 'scikit-learn is easy.'
    print(raw)

    default_nlp = load('en')
    text1 = default_nlp(raw)
    print('==>', [(t, t.pos_) for t in text1])

scikit-learn is easy.
==> [(scikit, 'NOUN'), (-, 'PUNCT'), (learn, 'NOUN'), (is, 'VERB'), (easy, 'ADJ'), (., 'PUNCT')]


결과를 보면, scikit-learn을 한 단어로 취급해주지 않았는데요. 'scikit-learn은 쉽다' 라는 말도안되는 소리를 넣어서 그런가요? <br>
아니죠. 패키지에게 우리의 의도를 알려줘야 분석도 우리 의도에 맞게 해주는 겁니다. <br><br>
여기서는 default로 내장된 토크나이저가 작동해서 저런 결과가 발생한 겁니다. 그렇다면, tokenizer를 새로 정의해주면 되겠는데요. <br>
spaCy가 요구하는 tokenizer형식이 callable입니다. 우리가 원하는 방식이 나오도록 Callable을 정의해봅시다 :

In [31]:
import re

class rTokenizer(object):
    def __init__(self, nlp):
        self.vocab = nlp.vocab

    def __call__(self, text):
        words = re.findall(pattern='[a-zA-Z][-a-zA-Z]+', string=text.lower())
        return Doc(self.vocab, words=words)

역시 __call__함수가 정의되어 있으니 Callable이죠. 임의로 그냥 rTokenizer라고 이름붙였습니다. 결과를 한번 볼까요 :

In [36]:
custom_nlp = load('en', create_make_doc=rTokenizer)
raw = 'scikit-learn is easy.'
print(raw)

text2 = custom_nlp(raw)
print('==>', [(t, t.pos_) for t in text2])

scikit-learn is easy.
==> [(scikit-learn, 'NOUN'), (is, 'VERB'), (easy, 'ADJ')]


# Appendix: Iterable

Callable과 관련된 얘기는 아니지만, Callable을 이해했다면, 이런 새로운 문제를 만났을때 해결이 쉽습니다. 아래는 scikit-learn의 tfidfVectorizer를 사용하는 예시인데요. 무슨 문제인지 알겠나요? :

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

corpus = ['동해~~물과 백두산이',
         '마~르고 닳~도록',
         '하느님이 보우하사',
         '우~리 나라 만세~']

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()

['나라', '도록', '동해', '르고', '만세', '물과', '백두산이', '보우하사', '하느님이']

지금 보면, TfIdfVectorizer의 feature들이 우리 의도와는 맞지 않게 토크나이징 됐습니다. 예를 들어, <b>동해~~물과</b>는 '동해', '물과' 2단어가 아닌 <b>'동해물과'</b> 한 단어로 해줘야 맞겠죠. 다행히 TfIdfVectorizer는 Iterable을 인자로 받습니다. 우리의 의도를 반영시킬 수 있다는 뜻입니다.<br>
그러면 토크나이저를 Iterable로 정의해서 넣어줘봅시다.

In [68]:
class rTokenizer(object):
    def __init__(self, sent):
        new_str = ''
        for c in sent:
            if c not in '~/,.':
                new_str += c
        self.tokens = new_str.split()
    def __iter__(self):
        for t in self.tokens:
            yield t
        
vectorizer = TfidfVectorizer(tokenizer=rTokenizer)
vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()

['나라', '닳도록', '동해물과', '마르고', '만세', '백두산이', '보우하사', '우리', '하느님이']

feature가 알맞게 됐네요. 이제 한번 단어 score를 볼까요. "바퀴를 재발명하지 마라."라는 격언처럼 rTokenizer를 재사용해서 corpus를 구성한 다음 점수를 구해봅니다 :

In [61]:
# -- Corpus 재구성
Toks = [ rTokenizer(sent) for sent in corpus ]
corpus = [[t for t in toks] for toks in Toks]
    
print('<corpus 확인>')
print( corpus )

# -- 단어 사전 만들기
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

print('\n<단어 점수 확인>')
for i, toks in enumerate(corpus):
    print('doc[%d]' % i)
    print( [ '%s(%.2f)'%(token, sp_matrix[i, word2id[token]]) for token in toks ] )

<corpus 확인>
[['동해물과', '백두산이'], ['마르고', '닳도록'], ['하느님이', '보우하사'], ['우리', '나라', '만세']]

<단어 점수 확인>
doc[0]
['동해물과(0.58)', '백두산이(0.58)']
doc[1]
['마르고(0.71)', '닳도록(0.71)']
doc[2]
['하느님이(0.71)', '보우하사(0.00)']
doc[3]
['우리(0.00)', '나라(0.71)', '만세(0.71)']


위처럼 너무 짧은 corpus에는 점수가 불분명한 것 같습니다. 조금만 더 긴 corpus로도 해볼게요. 점수가 잘 나오는지 봅시다. :

In [85]:
corpus = ['동해물과 백두산이 마르고 닳도록 하느님이 보우하사 우리나라 만세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '남산 위에 저 소나무 철갑을 두른 듯 바람서리 불변함은 우리 기상일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '가을 하늘 공활한데 높고 구름 없이 밝은 달은 우리 가슴 일편단심일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '이 기상과 이 맘으로 충성을 다하여 괴로우나 즐거우나 나라 사랑하세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '오 필승 코리아 오 필승 코리아 오 필승 코리아 오 오레 오레 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세']

class rTokenizer(object):
    def __init__(self, sent):
        new_str = ''
        for c in sent:
            if c not in '~/,.':
                new_str += c
        self.tokens = new_str.split()
    def __iter__(self):
        for t in self.tokens:
            yield t

vectorizer = TfidfVectorizer(tokenizer=rTokenizer)
sp_matrix = vectorizer.fit_transform(corpus)

# -- 단어 사전 만들기
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

# -- Corpus 재구성
Toks = [ rTokenizer(sent) for sent in corpus ]
corpus = [[t for t in toks ] for toks in Toks]
    
print('\n<단어 점수 확인>')
for i, toks in enumerate(corpus):
    print('doc[%d] : ' % i, end='')
    result = ' '.join([ '%s(%.2f)'%(token, sp_matrix[i, word2id[token]]) for token in toks ])
    print(result[:50], '' if len(result) < 50 else '...' )


<단어 점수 확인>
doc[0] : 동해물과(0.32) 백두산이(0.32) 마르고(0.32) 닳도록(0.32) 하느님이(0.3 ...
doc[1] : 남산(0.28) 위에(0.28) 저(0.28) 소나무(0.28) 철갑을(0.28) 두른(0 ...
doc[2] : 가을(0.28) 하늘(0.28) 공활한데(0.28) 높고(0.28) 구름(0.28) 없이( ...
doc[3] : 이(0.54) 기상과(0.27) 이(0.54) 맘으로(0.27) 충성을(0.27) 다하여( ...
doc[4] : 오(0.63) 필승(0.48) 코리아(0.48) 오(0.63) 필승(0.48) 코리아(0. ...


잘 나오긴 합니다. '오'의 점수가 높은데, '오'라는 단어가 4번 문서에서는 꽤 유의미하다고 해석되는 겁니다. 실제로 '오'라는 단어는 일반적으로 별로 필요없는 단어지만, 오~필승~코리아~라는 노래에서 만큼은 꽤 중요하긴 하죠. '오'만 들어도 노래가 떠오르니까요. Callable로 시작했는데 너무 많이 딴길로 와버렸네요. 아무튼 이렇게 파이썬을 엔지니어적으로 접근하는 것도 재미있으니 참고가 됐으면 합니다.